# <u>Setup</u>

In [47]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

import xgboost as xgb

import optuna

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# import torch.utils.tensorboard as tb

# import ray
# from ray import tune, train
# from ray.tune.schedulers import ASHAScheduler
# from ray.tune import CLIReporter
# ray.init(num_gpus=1, ignore_reinit_error=True) # ray needs explicit instructions to use GPU
# print(ray.cluster_resources())

# for feature extraction
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters

import matplotlib.pyplot as plt
import seaborn as sns

import os
import pickle
import random

In [57]:
## SAVE FEATURE STATISTICS FROM PREPROCESSING ## -- from the 1Hz processed data
X_MEAN = 21.234056
Y_MEAN = -16.93773
Z_MEAN = 24.948428

HR_MEAN = 81.085869 # bpm
WEIGHT_MEAN = 73.272422 # kg
HEIGHT_MEAN = 172.332927 # cm
GENDER_MEAN = 0.721742 # 0:female, 1:male -- represents class imbalance
AGE_MEAN = 30.236566 # yrs

EE_MEAN = 3481.625380 # cal/min (NOT kcal/min!!!!!)

X_STD = 35.144448
Y_STD = 37.25645
Z_STD = 23.991521

HR_STD = 29.852576
WEIGHT_STD = 11.258137
HEIGHT_STD = 6.570272
GENDER_STD = 0.448141
AGE_STD = 5.199545

EE_STD = 2752.199323

In [74]:
def resting_metabolic_rate(weight_kg):
    return weight_kg * 3.5 * 5 #cal/min

In [75]:
def mets(EE, weight_kg):
    return EE / resting_metabolic_rate(weight_kg)

In [76]:
def convert_to_mets(metrics_dict, convert_keys=['MAE', 'RMSE'], weight=WEIGHT_MEAN):
    converted_metrics = {}
    for key, value in metrics_dict.items():
        if key in convert_keys:
            converted_metrics[key] = mets(value, weight)
        else:
            converted_metrics[key] = value
    return converted_metrics

# <u>Model designs</u>

# <u>Load and prep data</u>

## Load data

In [3]:
def display_windows(subject_windows_list):
        display(subject_windows_list[0])
        display(subject_windows_list[-1])

In [4]:
windows_path = 'data/WEEE Dataset/processed/windows.pkl'
with open(windows_path, 'rb') as f:
    windows_dict = pickle.load(f)

windows_dict.keys()

dict_keys(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17'])

In [5]:
with open(windows_path, 'rb') as f:
    print(f.read(10))  # Read the first 10 bytes

b'\x80\x04\x95\x8d\x0f\x01\x00\x00\x00\x00'


In [6]:
display_windows(windows_dict['P01'])

,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,0.220972,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
1,0.220972,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
2,0.220972,0.454626,1.252592,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
3,0.249426,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
4,0.249426,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
...,...,...,...,...,...,...,...,...,...
187,0.249426,0.454626,1.252592,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203
188,0.249426,0.454626,1.294273,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203
189,0.249426,0.454626,1.294273,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203
190,0.249426,0.454626,1.294273,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203


,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,-0.063568,-0.780061,0.794096,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
1,-0.205838,-0.806902,0.544008,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
2,-0.120476,-0.780061,0.877459,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
3,-0.262746,-0.860584,0.544008,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
4,-0.006660,-0.887425,0.335601,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
...,...,...,...,...,...,...,...,...,...
187,-0.376562,-0.806902,0.627371,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886
188,0.021794,-0.833743,0.669052,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886
189,-0.177384,-0.967948,0.168875,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886
190,0.050248,-0.941107,0.377282,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886


# <u>Feature engineering</u>

## Extract features

In [20]:
def extract_temporal_feature_stats(
    windows_dict,
    subject_id,
    temporal_feature_cols = ['X', 'Y', 'Z', 'heart_rate'],
    label_column = 'EE[kcal/min]',
    custom_fc_parameters = {
        "sum_values": None,
        "median": None,
        "mean": None,
        "length": None,
        "standard_deviation": None,
        "variance": None,
        "root_mean_square": None,
        "maximum": None,
        "absolute_maximum": None,
        "minimum": None
    },
    testing = False
):
    temporal_features_list = []
    labels_list = []

    scale = 100 if testing else 1
    for i in range(len(windows_dict[subject_id])//scale):
        window_features = windows_dict[subject_id][i][temporal_feature_cols].copy()
        window_features['window_id'] = i
        temporal_features_list.append(window_features)

        labels = windows_dict[subject_id][i][label_column].copy().squeeze()
        labels_list.append(labels.mean())

    temporal_features_df = pd.concat(temporal_features_list)
    labels_series = pd.Series(labels_list)

    extracted_features = extract_features(temporal_features_df, column_id='window_id', default_fc_parameters=custom_fc_parameters).dropna(axis=1)

    return extracted_features, labels_series

In [21]:
# extracted_features, labels = extract_temporal_feature_stats(windows_dict, 'P01', testing=True)
# display(extracted_features)
# display(labels)

Feature Extraction: 100%|██████████| 20/20 [00:00<00:00, 2139.24it/s]


,X__sum_values,X__median,X__mean,X__length,X__standard_deviation,X__variance,X__root_mean_square,X__maximum,X__absolute_maximum,X__minimum,...,heart_rate__sum_values,heart_rate__median,heart_rate__mean,heart_rate__length,heart_rate__standard_deviation,heart_rate__variance,heart_rate__root_mean_square,heart_rate__maximum,heart_rate__absolute_maximum,heart_rate__minimum
0,46.808592,0.249426,0.243795,192.0,0.011337,0.000129,0.244058,0.249426,0.249426,0.220972,...,83.025767,0.452786,0.432426,192.0,0.059374,0.003525,0.436483,0.499460,0.499460,0.322391
1,47.491488,0.249426,0.247352,192.0,0.007398,0.000055,0.247462,0.249426,0.249426,0.220972,...,74.232845,0.386120,0.386629,192.0,0.082985,0.006886,0.395435,0.478380,0.478380,0.282831
2,47.804482,0.249426,0.248982,192.0,0.003529,0.000012,0.249007,0.249426,0.249426,0.220972,...,76.998356,0.437988,0.401033,192.0,0.076099,0.005791,0.408189,0.478380,0.478380,0.282831
3,47.804482,0.249426,0.248982,192.0,0.003529,0.000012,0.249007,0.249426,0.249426,0.220972,...,87.557008,0.451565,0.456026,192.0,0.058311,0.003400,0.459739,0.571102,0.571102,0.377621
4,48.487379,0.249426,0.252538,192.0,0.011002,0.000121,0.252778,0.277880,0.277880,0.220972,...,96.628647,0.537069,0.503274,192.0,0.067083,0.004500,0.507725,0.571102,0.571102,0.377621


0   -0.396848
1   -0.457849
2   -0.566454
3   -0.568399
4   -0.541365
dtype: float64

In [25]:
def feature_engineering(windows_dict, label_column='EE[kcal/min]', demographic_columns=['Weight', 'Height', 'Gender', 'Age']):
    """
    Prepares the temporal and demographic features and labels as pandas DataFrames 
    for leave-one-subject-out cross-validation.

    Args:
        windows_dict: Dictionary of preprocessed windows.
        label_column: The column to extract as labels. Note: The label units are cal/min, NOT kcal/min as stated above.
        demographic_columns: List of columns representing demographic features.

    Returns:
        data_dict: A dictionary with subject-wise splits for temporal features, 
                   demographic features, and labels as pandas DataFrames.
    """
    data_dict = {}
    custom_fc_parameters = {
        "sum_values": None,
        "median": None,
        "mean": None,
        "length": None,
        "standard_deviation": None,
        "variance": None,
        "root_mean_square": None,
        "maximum": None,
        "absolute_maximum": None,
        "minimum": None
    }

    for subject_id, windows in windows_dict.items():
        temporal_features_df, labels = extract_temporal_feature_stats(
            windows_dict, 
            subject_id, 
            temporal_feature_cols = ['X', 'Y', 'Z', 'heart_rate'],
            label_column=label_column, 
            custom_fc_parameters=custom_fc_parameters
        )

        constant_features = []
        for window in windows:
            constant_array = window[demographic_columns].iloc[0].values  # Single row for demographic data
            constant_features.append(constant_array)  # Shape: (num_constant_features,)

        constant_features_df = pd.DataFrame(constant_features, columns=demographic_columns)

        data_dict[subject_id] = {
            'temporal_features': temporal_features_df,
            'constant_features': constant_features_df,
            'labels': labels 
        }

    return data_dict

In [22]:
data_dict = feature_engineering(windows_dict)
data_dict

Feature Extraction: 100%|██████████| 30/30 [00:00<00:00, 182.67it/s]


{'P01': {'temporal_features':      X__sum_values  X__median   X__mean  X__length  X__standard_deviation  \
  0        46.808592   0.249426  0.243795      192.0               0.011337   
  1        47.491488   0.249426  0.247352      192.0               0.007398   
  2        47.804482   0.249426  0.248982      192.0               0.003529   
  3        47.804482   0.249426  0.248982      192.0               0.003529   
  4        48.487379   0.249426  0.252538      192.0               0.011002   
  ..             ...        ...       ...        ...                    ...   
  593     -42.850002  -0.234292 -0.223177      192.0               0.110349   
  594     -35.935677  -0.205838 -0.187165      192.0               0.113432   
  595     -37.273015  -0.205838 -0.194130      192.0               0.151687   
  596     -33.346361  -0.205838 -0.173679      192.0               0.192610   
  597     -26.375128  -0.148930 -0.137370      192.0               0.201810   
  
       X__variance  X

### Test

In [16]:
data_dict['P01'].keys()

dict_keys(['temporal_features', 'constant_features', 'labels'])

In [23]:
data_dict['P01']['temporal_features']

,X__sum_values,X__median,X__mean,X__length,X__standard_deviation,X__variance,X__root_mean_square,X__maximum,X__absolute_maximum,X__minimum,...,heart_rate__sum_values,heart_rate__median,heart_rate__mean,heart_rate__length,heart_rate__standard_deviation,heart_rate__variance,heart_rate__root_mean_square,heart_rate__maximum,heart_rate__absolute_maximum,heart_rate__minimum
0,46.808592,0.249426,0.243795,192.0,0.011337,0.000129,0.244058,0.249426,0.249426,0.220972,...,83.025767,0.452786,0.432426,192.0,0.059374,0.003525,0.436483,0.499460,0.499460,0.322391
1,47.491488,0.249426,0.247352,192.0,0.007398,0.000055,0.247462,0.249426,0.249426,0.220972,...,74.232845,0.386120,0.386629,192.0,0.082985,0.006886,0.395435,0.478380,0.478380,0.282831
2,47.804482,0.249426,0.248982,192.0,0.003529,0.000012,0.249007,0.249426,0.249426,0.220972,...,76.998356,0.437988,0.401033,192.0,0.076099,0.005791,0.408189,0.478380,0.478380,0.282831
3,47.804482,0.249426,0.248982,192.0,0.003529,0.000012,0.249007,0.249426,0.249426,0.220972,...,87.557008,0.451565,0.456026,192.0,0.058311,0.003400,0.459739,0.571102,0.571102,0.377621
4,48.487379,0.249426,0.252538,192.0,0.011002,0.000121,0.252778,0.277880,0.277880,0.220972,...,96.628647,0.537069,0.503274,192.0,0.067083,0.004500,0.507725,0.571102,0.571102,0.377621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,-42.850002,-0.234292,-0.223177,192.0,0.110349,0.012177,0.248968,0.192518,0.461924,-0.461924,...,117.140226,0.500911,0.610105,192.0,0.381287,0.145380,0.719450,1.265690,1.265690,0.208530
594,-35.935677,-0.205838,-0.187165,192.0,0.113432,0.012867,0.218855,0.164064,0.405016,-0.405016,...,127.814055,0.543162,0.665698,192.0,0.350520,0.122864,0.752342,1.265690,1.265690,0.235395
595,-37.273015,-0.205838,-0.194130,192.0,0.151687,0.023009,0.246365,0.363242,0.490378,-0.490378,...,207.369507,0.796306,1.080050,192.0,0.675266,0.455984,1.273770,2.082661,2.082661,0.431106
596,-33.346361,-0.205838,-0.173679,192.0,0.192610,0.037099,0.259351,0.391696,0.518832,-0.518832,...,364.753545,2.016529,1.899758,192.0,0.689695,0.475679,2.021079,2.618413,2.618413,0.458496


In [17]:
data_dict['P01']['constant_features']

,Weight,Height,Gender,Age
0,-0.237377,-0.963877,0.0,-0.237822
1,-0.237377,-0.963877,0.0,-0.237822
2,-0.237377,-0.963877,0.0,-0.237822
3,-0.237377,-0.963877,0.0,-0.237822
4,-0.237377,-0.963877,0.0,-0.237822
...,...,...,...,...
593,-0.237377,-0.963877,0.0,-0.237822
594,-0.237377,-0.963877,0.0,-0.237822
595,-0.237377,-0.963877,0.0,-0.237822
596,-0.237377,-0.963877,0.0,-0.237822


In [24]:
data_dict['P01']['labels']

0     -0.396848
1     -0.457849
2     -0.566454
3     -0.568399
4     -0.541365
         ...   
593    0.520095
594    0.716404
595    1.218670
596    0.731687
597    0.442454
Length: 598, dtype: float64

## Drop subjects

In [28]:
def drop_subjects(data_dict, exclude_keys=['P03', 'P04', 'P06', 'P11', 'P13', 'P14']):
    """
    Drops subjects from the data dictionary.

    Args:
        data_dict: Dictionary of subject-wise data.
        exclude_keys: List of subject IDs to exclude.

    Returns:
        Dictionary with specified subjects removed.
    """
    return {k: v for k, v in data_dict.items() if k not in exclude_keys}

## LOO-CV Split

In [27]:
def leave_one_out_split(data_dict, exclude_keys=['P03', 'P04', 'P06', 'P11', 'P13', 'P14']):
    """
    Generator for leave-one-subject-out cross-validation splits with temporal and constant features.

    Args:
        data_dict: Dictionary with subject-wise temporal features, constant features, and labels as pandas DataFrames.

    Yields:
        train_data: Tuple of training temporal features, constant features, and labels as pandas DataFrames.
        test_data: Tuple of testing temporal features, constant features, and labels as pandas DataFrames.
    """
    data_dict = {k: v for k, v in data_dict.items() if k not in exclude_keys}
    subjects = list(data_dict.keys())

    for test_subject in subjects:
        train_temporal_features = []
        train_constant_features = []
        train_labels = []
        
        for subject, data in data_dict.items():
            if subject == test_subject:
                test_temporal_features = data['temporal_features']
                test_constant_features = data['constant_features']
                test_labels = data['labels']
            else:
                train_temporal_features.append(data['temporal_features'])
                train_constant_features.append(data['constant_features'])
                train_labels.append(data['labels'])

        # Combine training data using pandas DataFrames
        train_temporal_features = pd.concat(train_temporal_features, axis=0)
        train_constant_features = pd.concat(train_constant_features, axis=0)
        train_labels = pd.concat(train_labels, axis=0)

        train_data = (train_temporal_features, train_constant_features, train_labels)
        test_data = (test_temporal_features, test_constant_features, test_labels)

        yield train_data, test_data, test_subject

### Test

In [29]:
sorted(data_dict.keys())

['P01',
 'P02',
 'P03',
 'P04',
 'P05',
 'P06',
 'P07',
 'P08',
 'P09',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17']

In [30]:
loocv_splits = leave_one_out_split(data_dict, exclude_keys=[]) # turned off subject dropping
loocv_splits

<generator object leave_one_out_split at 0x73c11d911f50>

In [31]:
loocv_test = next(loocv_splits)

In [32]:
len(loocv_test)

3

In [33]:
loocv_test[0][0].shape, loocv_test[0][1].shape, loocv_test[0][2].shape

((8657, 40), (8657, 4), (8657,))

In [34]:
loocv_test[1][0].shape, loocv_test[1][1].shape, loocv_test[1][2].shape

((598, 40), (598, 4), (598,))

In [35]:
loocv_test[2]

'P01'

## Train Test Split

In [36]:
def split_data_dict(data_dict, test_size=0.2, random_state=42):
    """
    Splits the data_dict into training and testing sets regardless of subject_id.

    Args:
        data_dict: Dictionary with subject-wise temporal features, constant features, and labels as pandas DataFrames.
        test_size: Proportion of the dataset to include in the test split.
        random_state: Controls the shuffling applied to the data before applying the split.

    Returns:
        train_data: Tuple of training temporal features, constant features, and labels as pandas DataFrames.
        test_data: Tuple of testing temporal features, constant features, and labels as pandas DataFrames.
    """
    all_temporal_features = []
    all_constant_features = []
    all_labels = []

    for data in data_dict.values():
        all_temporal_features.append(data['temporal_features'])
        all_constant_features.append(data['constant_features'])
        all_labels.append(data['labels'])

    all_temporal_features = pd.concat(all_temporal_features, axis=0)
    all_constant_features = pd.concat(all_constant_features, axis=0)
    all_labels = pd.concat(all_labels, axis=0)

    temporal_train, temporal_test, constant_train, constant_test, labels_train, labels_test = train_test_split(
        all_temporal_features, all_constant_features, all_labels, test_size=test_size, random_state=random_state
    )

    train_data = (temporal_train, constant_train, labels_train)
    test_data = (temporal_test, constant_test, labels_test)

    return train_data, test_data

# <u>Train Models</u>

## Training functions

In [51]:
def unstandardize_predictions(predictions, mean=EE_MEAN, std=EE_STD):
    """
    Unstandardize the predictions back to the original scale.

    Args:
        predictions: Standardized predictions.
        mean: Mean of the original data.
        std: Standard deviation of the original data.

    Returns:
        np.ndarray: Unstandardized predictions.
    """
    return predictions * std + mean

In [58]:
def train_evaluate_xgboost(train_data, test_data, param=None):
    """
    Train and evaluate an XGBoost model on the provided train and test data.

    Args:
        train_data: Tuple of training temporal features, constant features, and labels as pandas DataFrames.
        test_data: Tuple of testing temporal features, constant features, and labels as pandas DataFrames.
        param: Dictionary of hyperparameters for the XGBoost model.

    Returns:
        dict: A dictionary containing the evaluation metrics (MAE (cal/min), RMSE (cal/min), R^2).
    """
    train_temporal_features, train_constant_features, train_labels = train_data
    test_temporal_features, test_constant_features, test_labels = test_data

    # Concatenate temporal and constant features
    X_train = pd.concat([train_temporal_features, train_constant_features], axis=1)
    X_test = pd.concat([test_temporal_features, test_constant_features], axis=1)
    y_train = train_labels
    y_test = test_labels

    # Default parameters if none are provided
    if param is None:
        param = {
            'objective': 'reg:squarederror',
            'n_estimators': 100,
            'learning_rate': 0.1,
            'max_depth': 6
        }

    # Train XGBoost model
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Unstandardize predictions
    y_pred_unstandardized = unstandardize_predictions(y_pred)
    y_test_unstandardized = unstandardize_predictions(y_test)

    # Evaluate metrics
    mae = mean_absolute_error(y_test_unstandardized, y_pred_unstandardized)
    rmse = np.sqrt(mean_squared_error(y_test_unstandardized, y_pred_unstandardized))
    r2 = r2_score(y_test_unstandardized, y_pred_unstandardized)

    metrics = {
        'MAE': mae,
        'RMSE': rmse,
        'R^2': r2
    }

    return metrics

# # Example usage with leave_one_out_split
# loocv_splits = leave_one_out_split(data_dict, exclude_keys=[])
# train_data, test_data, test_subject = next(loocv_splits)
# metrics = train_evaluate_xgboost(train_data, test_data)
# print(f"Evaluation metrics for subject {test_subject}: {metrics}")

# # Example usage with split_data_dict
# train_data, test_data = split_data_dict(data_dict)
# metrics = train_evaluate_xgboost(train_data, test_data)
# print(f"Evaluation metrics: {metrics}")

In [59]:
def loocv_train_eval_xgboost(data_dict, exclude_keys=[]):
    """
    Trains and evaluates an XGBoost model using leave-one-out cross-validation.

    Args:
        data_dict: Dictionary with subject-wise temporal features, constant features, and labels as pandas DataFrames.
        exclude_keys: List of subject IDs to exclude from the cross-validation.

    Returns:
        evaluation_df: DataFrame containing evaluation metrics (MAE (cal/min), RMSE (cal/min), R^2) for each subject.
    """
    evaluation_results = []

    loocv_splits = leave_one_out_split(data_dict, exclude_keys=exclude_keys)

    for train_data, test_data, test_subject in loocv_splits:
        metrics = train_evaluate_xgboost(train_data, test_data)
        metrics['Subject'] = test_subject
        evaluation_results.append(metrics)

    evaluation_df = pd.DataFrame(evaluation_results)

    return evaluation_df

In [54]:
loocv_eval_results = loocv_train_eval_xgboost(data_dict)
loocv_eval_results

,MAE,RMSE,R^2,Subject
0,923.160310,1186.167383,0.715201,P01
1,1351.408754,1861.724916,0.693472,P02
2,983.043043,1446.733798,-0.354673,P03
3,776.120138,1154.150550,0.612162,P04
4,991.940181,1535.277564,0.628813,P05
5,2411.844869,2886.888407,-4.534806,P06
6,922.712291,1221.196742,0.634246,P07
7,1228.512351,1719.245416,0.623293,P08
8,889.151599,1372.716659,0.778822,P09
9,1112.807268,1460.677981,0.351542,P10


In [60]:
def report_avg_loocv_results(evaluation_df):
    """
    Reports the average evaluation metrics for leave-one-out cross-validation.

    Args:
        evaluation_df: DataFrame containing evaluation metrics (MAE (cal/min), RMSE (cal/min), R^2) for each subject.

    Returns:
        dict: A dictionary containing the average evaluation metrics.
    """
    avg_metrics = evaluation_df.drop(columns=['Subject']).mean()
    return avg_metrics.to_dict()

In [56]:
report_avg_loocv_results(loocv_eval_results)

{'MAE': 1195.9786144040393,
 'RMSE': 1677.9714154631283,
 'R^2': 0.17522307592925784}

## Tuning

In [49]:
def objective(trial):
    # Define the hyperparameters to tune
    param = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    # Perform LOOCV
    loocv_splits = leave_one_out_split(data_dict)
    rmses = []

    for train_data, test_data, _ in loocv_splits:
        metrics = train_evaluate_xgboost(train_data, test_data, param)
        rmses.append(metrics['RMSE'])

    # Return the average RMSE as the objective value
    return np.mean(rmses)

def optimize_xgboost(data_dict, n_trials=50):
    """
    Optimize XGBoost hyperparameters using Optuna and evaluate the performance.

    Args:
        data_dict: Dictionary with subject-wise temporal features, constant features, and labels as pandas DataFrames.
        n_trials: Number of trials for the optimization.

    Returns:
        best_params: The best hyperparameters found by Optuna.
        best_metrics: The evaluation metrics for the best model.
    """
    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    # Get the best hyperparameters
    best_params = study.best_params

    # Evaluate the model with the best hyperparameters using LOOCV
    loocv_eval_results = loocv_train_eval_xgboost(data_dict, exclude_keys=[])
    best_metrics = report_avg_loocv_results(loocv_eval_results)

    return best_params, best_metrics

# # Example usage
# best_params, best_metrics = optimize_xgboost(data_dict, n_trials=50)
# print(f"Best hyperparameters: {best_params}")
# print(f"Best evaluation metrics: {best_metrics}")

In [65]:
best_params, best_metrics = optimize_xgboost(data_dict, n_trials=50)

[I 2025-01-10 17:35:40,632] A new study created in memory with name: no-name-9c459205-aa1a-4b54-a051-bc2c8c65e1f7
[I 2025-01-10 17:35:53,883] Trial 0 finished with value: 1644.4700804670108 and parameters: {'n_estimators': 112, 'learning_rate': 0.2622831087509353, 'max_depth': 9, 'min_child_weight': 2, 'subsample': 0.9736198496754798, 'colsample_bytree': 0.5656771006922577}. Best is trial 0 with value: 1644.4700804670108.
[I 2025-01-10 17:36:02,161] Trial 1 finished with value: 1730.0741947597555 and parameters: {'n_estimators': 166, 'learning_rate': 0.1798235661057978, 'max_depth': 5, 'min_child_weight': 4, 'subsample': 0.7071586616695132, 'colsample_bytree': 0.6970269782926564}. Best is trial 0 with value: 1644.4700804670108.
[I 2025-01-10 17:36:08,286] Trial 2 finished with value: 1593.9639216614698 and parameters: {'n_estimators': 134, 'learning_rate': 0.06358039144819068, 'max_depth': 4, 'min_child_weight': 6, 'subsample': 0.7830408377498623, 'colsample_bytree': 0.7205391617462511

Best hyperparameters: {'n_estimators': 181, 'learning_rate': 0.02106018884749126, 'max_depth': 5, 'min_child_weight': 4, 'subsample': 0.5282414747317894, 'colsample_bytree': 0.8038432142555246}
Best evaluation metrics cal/min: {'MAE': 1195.9786144040393, 'RMSE': 1677.9714154631283, 'R^2': 0.17522307592925784}
Best evaluation metrics METs: {'MAE': 932.706102014783, 'RMSE': 1308.5971265370117, 'R^2': 0.13665096529711532}


In [77]:
print(f"Best hyperparameters: {best_params}")
print(f"Best evaluation metrics cal/min: {best_metrics}")
print(f"Best evaluation metrics METs: {convert_to_mets(best_metrics)}")

Best hyperparameters: {'n_estimators': 181, 'learning_rate': 0.02106018884749126, 'max_depth': 5, 'min_child_weight': 4, 'subsample': 0.5282414747317894, 'colsample_bytree': 0.8038432142555246}
Best evaluation metrics cal/min: {'MAE': 727.3881508066142, 'RMSE': 1020.5337372259621, 'R^2': 0.10656978949606154}
Best evaluation metrics METs: {'MAE': 0.5672671389100442, 'RMSE': 0.7958821609004444, 'R^2': 0.10656978949606154}
